# Kartenvisualisierung
In diesem Notebook wird demonstriert, wie in Python eine Kartenvisualisierung möglich ist.

Wir stützen uns dabei auf die Tabelle, die im xml_to_table-Skript erstellt wurde.

## Koordinaten hinzufügen
Da unsere Tabelle zwar die Keys der Orte enthält, aber nicht die IDs, müssen wir nochmal die API bemühen, um die Koordinaten zu erhalten.

In [ ]:
import pandas as pd

# Einlesen der Tabelle
df = pd.read_csv('places.csv')

print(df.head)

In [ ]:
# wir wollen nicht Orte mehrmals abfragen, auch um den Editions-Server zu schonen
# also machen wir erstmal eine Liste aller einzigartigen Orts-Keys
unique_place_keys = df['place_key'].unique().tolist()

print(unique_place_keys)

In [ ]:
# Abfrage der API
import requests
from lxml import etree as et

# wir möchten ein mapping erstellen, in dem für jeden Key eine Latitude und eine Longitude Koordinate gefunden werden.
key_to_coordinates = {}

for place_key in unique_place_keys:
    response = requests.get("https://sturm-edition.de/api/places/" + place_key)
    xml = et.fromstring(response.text)
    location = xml.xpath("./tei:location/tei:geo", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})[0]
    if location is not None:
        latitude, longitude = location.text.split()
        key_to_coordinates[place_key] = (latitude, longitude)
    else:
        # falls keine Koordinaten hinterlegt sind
        key_to_coordinates[place_key] = (None, None)

print(key_to_coordinates)

In [ ]:
# Zurückführen der Koordinaten in Tabelle
df[['latitude', 'longitude']] = df['place_key'].map(key_to_coordinates).apply(pd.Series)

print(df)

## Interaktive Karten mit Python
Im Folgenden verwenden wir das externe Package [*Folium*](https://python-visualization.github.io/folium/latest/index.html), um unsere Orte in einer Karte darzustellen.

In [ ]:
%pip install -U folium

In [ ]:
import folium

# Eine Tabelle nur mit den relevanten Informationen für die Visualisierung
places_df = df[['place_name', 'place_key', 'latitude', 'longitude']].drop_duplicates()

# die Koordinaten sollten als Zahlenwerte vorliegen, wir stellen das hier nochmal sicher
places_df['latitude'] = places_df['latitude'].astype(float)
places_df['longitude'] = places_df['longitude'].astype(float)

# Initialisieren der Karte mit Einstellungen, wo anfangs der Ausschnitt liegen soll
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Unsere Orte auf der Karte anzeigen
for _, row in places_df.iterrows():
    # wir erstellen einen Marker pro Zeile in diesem Fall
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row["place_name"],
        tooltip=row['place_name'],
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(m)

# Display map
m

### Häufigkeit repräsentieren

In [ ]:
# wir zählen wie oft jeder Ort vorkommt
place_counts = df['place_key'].value_counts().rename_axis('place_key').reset_index(name='count')

# Merge zwischen dem reduzierten df und unserer Zählung
places_df = places_df.merge(place_counts, on='place_key')

print(places_df)

In [ ]:
m = folium.Map(location=[51.0, 10.0], zoom_start=4)

for _, row in places_df.iterrows():
    # Wir vergrössern die Marker, falls der Ort häufiger vorkommt, aber setzen eine Maximalgrösse von 30
    marker_size = min(10 + row["count"] * 2, 30)
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=marker_size,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=f"{row['place_name']}<br>Vorkommen: {row['count']}",
        tooltip=row['place_name']
    ).add_to(m)

# Anzeigen der Karte
m

### Visualisierung über Zeit
Beachte hier, dass Folium bzw. TimestampedGeoJson leider keine Datumsangaben vor 1970 anzeigen kann, deshalb behelfen wir uns, indem wir alle Datumsangaben 100 Jahre nach hinten rücken.

In [ ]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# Sicherstellen, dass Datumsangaben und Koordinaten im richtigen Format sind
df['date'] = pd.to_datetime(df['date'])
df['date_fitted'] = df['date'].apply(lambda x: x.replace(year=x.year + 100))

df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

# Alle Einträge ohne Datumsangabe entfernen
df = df.dropna(subset=['date'])

# Zeilen nach Datum sortieren
df = df.sort_values(by='date')

# Unsere Daten in GeoJSON-Format schreiben
geojson_features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row["longitude"], row["latitude"]]
        },
        "properties": {
            "times": [row["date_fitted"].isoformat()],  # Convert to timestamp format
            "popup": f"{row['place_name']}<br>Datierung: {row['date']}",
            "tooltip": row["place_name"]
        }
    }
    geojson_features.append(feature)

m = folium.Map(location=[51.0, 10.0], zoom_start=4)

# Die GeoJSON-Infos der Karte hinzufügen
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": geojson_features
    },
    period="P1M",  # Jeder Timestep = 1 Monat
    duration="P6M",  # Punkte bleiben für 6 Monate danach noch sichtbar
    auto_play=False  # Auto-Play abschalten
).add_to(m)

# Karte anzeigen
m

# Die Karte als Datei speichern, die wir im Browser öffnen können.
m.save("time_map.html")